In [1]:
import gc, functools,time
import pickle, re
import random
from typing import List
from pprint import pprint

In [2]:
from pyecharts.charts import Bar, Tab
from pyecharts import options as opts

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.spatial import distance
sns.set(style="whitegrid")
import gensim
import jieba
from tqdm import tqdm
from gensim.models.ldamodel import LdaModel


C:\Users\gaowenxin95\AppData\Roaming\Python\Python37\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
stopwords = open('../data/cn_stopwords.txt',encoding='utf-8').read().split()
stopwords = frozenset(stopwords)
print(len(stopwords))

746


In [5]:
dataset = pd.read_csv('../data/nCoV_100k_train.labled.csv',encoding='utf-8').sample(n=1000)
dataset.head()

,微博id,微博发布时间,发布人账号,微博中文内容,微博图片,微博视频,情感倾向
68869,4472377041362250,02月15日 23:40,himepis1,劝告自己一句话不要阴谋论?,[],[],-1
76275,4468736511602610,02月05日 22:34,上局沪段_沪,达到这个层面若再耐热搞不好就是全人类的问题了……//@急诊夜鹰:病毒中的全能冠军？不可能吧/...,['https://ww4.sinaimg.cn/orj360/9e5389bbly1gbl...,[],0
86563,4472198762395460,02月15日 11:52,希来欢微巧,【连康山保护区】地带性植被属常绿，落叶阔叶混交林。现仅存黄山松和马尾松与栎类的天然次生林，动...,['https://ww2.sinaimg.cn/orj360/005KAoOdgy1gbw...,[],0
83910,4464030725396240,01月23日 22:55,真实爆光天使迖人,1月22日，有媒体接到了一位武汉的哥打来的电话，他说接了一个护士到金银潭医院，这名护士是主动...,['https://ww2.sinaimg.cn/orj360/006DBT8Jgy1gb6...,[],1
28577,4469481533048160,02月07日 23:54,爱吃爱长腿爱jay,哈哈哈太形象了,['https://imgaliyuncdn.miaopai.com/images/8gbW...,['https://miaopai.video.weibocdn.com/004C3KrYl...,1


In [46]:
def seg_words(text):
    #去掉不在(所有中文、大小写字母、数字)中的非法字符
    regex = re.compile(r'[^\u4e00-\u9fa5]')
    text = regex.sub('', text) # 将非法字符用‘ ’替代
    text = text.strip() # 去掉前后的空格
   # text = text.replace(' ','')
    word_list = jieba.cut(text, cut_all= False)
    return word_list

In [60]:
train = dataset['微博中文内容'].apply(lambda x: "".join(seg_words(str(x))))

In [61]:
# docs = []
# #df_docs = dataset[dataset['微博中文内容'].notnull()]
# for row_val in train:
#     val = str(row_val).split('，')
#     docs.append(' '.join(val))
    
# len(docs)

In [62]:

def clean_words(s):
    return list(filter(lambda x: x not in stopwords and x != '[\u4e00-\u9fa5]', jieba.cut(s)))

In [63]:
corpus = []
for i in tqdm(train):
    corpus.append(clean_words(i))

100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 3070.55it/s]


In [64]:
for c in tqdm(corpus):
    while '^[\u4E00-\u9FFF]+$' in c:
        c.remove('^[\u4E00-\u9FFF]+$')

100%|█████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 1161535.31it/s]


In [65]:
print(f'文档数量{len(corpus)}, 总词数{sum(len(c) for c in corpus)}')

文档数量1000, 总词数28452


In [66]:
dictionary = gensim.corpora.Dictionary(corpus)
doc_term_matrix = [dictionary.doc2bow(i) for i in corpus]

In [67]:
%%time
lda = LdaModel(doc_term_matrix, id2word=dictionary, chunksize=8000,
    num_topics=10, random_state=2020)

Wall time: 1.19 s


In [68]:
def extract(k):
    return [[lda.id2word[i], j] for i, j in lda.get_topic_terms(k, topn=30)]

In [69]:
for i in range(10):
    pprint(extract(i))
    print()

[['肺炎', 0.005982815],
 ['疫情', 0.0057649757],
 ['病毒', 0.00575671],
 ['新型', 0.00528589],
 ['一个', 0.0052763787],
 ['加油', 0.0050953934],
 ['武汉', 0.004919909],
 ['展开', 0.004786764],
 ['中国', 0.0043846816],
 ['一起', 0.0042773033],
 ['网页', 0.0038465029],
 ['链接', 0.0037196889],
 ['全文', 0.0036978782],
 ['冠状病毒', 0.003208782],
 ['没有', 0.0031658032],
 ['行动', 0.00316375],
 ['月', 0.0029890742],
 ['微博', 0.002880169],
 ['视频', 0.0028076305],
 ['感染', 0.0025355152],
 ['社区', 0.0023703943],
 ['口罩', 0.002296156],
 ['说', 0.0022555469],
 ['生活', 0.0021906274],
 ['希望', 0.0021159844],
 ['病例', 0.0019430169],
 ['年', 0.0019287061],
 ['可能', 0.0019285843],
 ['带', 0.0018342704],
 ['战疫', 0.0018120033]]

[['武汉', 0.010638502],
 ['加油', 0.0066801333],
 ['疫情', 0.0053905626],
 ['全文', 0.0042967014],
 ['中国', 0.0037111186],
 ['一定', 0.0033381223],
 ['展开', 0.003274233],
 ['口罩', 0.003206164],
 ['医院', 0.002754623],
 ['湖北', 0.0027413007],
 ['真的', 0.0026743647],
 ['没有', 0.0024249698],
 ['西安', 0.0024179418],
 ['希望', 0.0024140638],
 ['会'

In [70]:
from pyecharts.charts import Bar, Tab
from pyecharts import options as opts
def get_tab(lda: LdaModel) -> Tab:
    tab = Tab()
    for i in range(lda.num_topics):
        words = [lda.id2word[i[0]]
            for i in lda.get_topic_terms(i, topn=10)]
        val = [i[1] for i in lda.get_topic_terms(i, topn=10)]
        bar = (
            Bar(opts.InitOpts(width="1000px"))
            .add_xaxis(words)
            .add_yaxis("权重 % ", [round(k*100, 3) for k in val])
            .set_global_opts(title_opts=opts.TitleOpts(title=f"主题 {i+1}"))
            )
        tab.add(bar, f'主题{i+1}')
    return tab

In [71]:
tab = get_tab(lda)

In [72]:
tab.render_notebook()